# OpenAI Quickstart Guide

From the official documentation available at: https://platform.openai.com/docs/overview

You can use different models depending on your needs, check them at: https://platform.openai.com/docs/models

Note: check the pricing before using a model! --> https://platform.openai.com/docs/pricing

In [1]:
# Import required libraries
from dotenv import load_dotenv
import os
import openai
from pprint import pprint

# Load environment variables from .env file
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

# Check if the API key is set
if not api_key:
    raise ValueError("OPENAI_API_KEY environment variable is not set.")

# Set OpenAI API key
openai.api_key = api_key

# Initialize OpenAI client
from openai import OpenAI
client = OpenAI()

In [ ]:
# Make a request to the model
response = client.responses.create(
    model="gpt-4.1",
    input="Write a one-sentence bedtime story about a unicorn."
)

# Print the model's response
print("Response = ")
pprint(dict(response))
print(f"\nresponse.output_text = \n{response.output_text}")

You can specify instructions to provide high-level instructions adopted as an overall context for your prompts.

In [8]:
# Example of using the 'instructions' and 'reasoning' parameters in the model request
response_with_instructions = client.responses.create(
    model="gpt-4.1",
    input="Write a poem about a butterfly.",
    instructions="Answer in rhyme and with a cheerful style.")

# With some models you can also specify the reasoning effort parameter, e.g.    
# reasoning={"effort": "low"})
  
print("Response with instructions = ")
pprint(dict(response_with_instructions))
print(f"\nresponse_with_instructions.output_text = \n{response_with_instructions.output_text}")

Response with instructions = 
{'background': False,
 'conversation': None,
 'created_at': 1756760035.0,
 'error': None,
 'id': 'resp_68b607e34f208197ab5c49642b94e1840d7e0e684004ad84',
 'incomplete_details': None,
 'instructions': 'Answer in rhyme and with a cheerful style.',
 'max_output_tokens': None,
 'max_tool_calls': None,
 'metadata': {},
 'model': 'gpt-4.1-2025-04-14',
 'object': 'response',
 'output': [ResponseOutputMessage(id='msg_68b607e3e97c8197abf77607bb79c85d0d7e0e684004ad84', content=[ResponseOutputText(annotations=[], text='A butterfly flutters in sunlight so bright,  \nWings painted with splashes of pure delight.  \nIt dances on breezes, a soft, gentle flight,  \nA whisper of wonder, a beautiful sight.\n\nFrom blossom to blossom it skips on the way,  \nPainting the garden with colors of May.  \nIt sips from the nectar, then giggles with glee,  \nA speckled-winged artist as light as can be.\n\nWith silken wings open, it glimmers and spins,  \nA jewel on a breeze where the

## Structured Output

Structured Output allows you to receive responses from the model in a predefined format, such as JSON or other structured data types. This is useful when you need the model's output to be machine-readable for further processing, integration, or automation. By specifying the desired structure, you can ensure consistency and make it easier to extract specific information from the model's response.

In [ ]:
from openai import OpenAI
from pydantic import BaseModel

client = OpenAI()

class Step(BaseModel):
    explanation: str
    output: str

class MathReasoning(BaseModel):
    steps: list[Step]
    final_answer: str

response = client.responses.parse(
    model="gpt-5-mini",
    input=[
        {
            "role": "system",
            "content": "You are a helpful math tutor. Guide the user through the solution step by step.",
        },
        {"role": "user", "content": "how can I solve 8x + 7 = -23"},
    ],
    text_format=MathReasoning,
)

math_reasoning = dict(response.output_parsed)
print("math_reasoning = \n")
pprint(math_reasoning)

math_reasoning = 

{'final_answer': 'x = -15/4 (which equals -3.75)',
 'steps': [Step(explanation='Start with the equation and isolate the term containing x by subtracting 7 from both sides.', output='8x + 7 = -23  ⇒  8x = -23 - 7 = -30'),
           Step(explanation='Now solve for x by dividing both sides by 8.', output='x = -30/8'),
           Step(explanation='Simplify the fraction by dividing numerator and denominator by 2.', output='x = -15/4 = -3.75'),
           Step(explanation='Check the solution by substituting x back into the original equation: compute 8x + 7 with x = -15/4.', output='8(−15/4) + 7 = −30 + 7 = −23, which matches the original right-hand side.')]}


Or again you can use structured output to request the response to be in a specific format, e.g:

In [8]:
from openai import OpenAI
from pydantic import BaseModel

client = OpenAI()

class CalendarEvent(BaseModel):
    name: str
    date: str
    participants: list[str]

response = client.responses.parse(
    model="gpt-4.1-mini",
    input=[
        {"role": "system", "content": "Extract the event information."},
        {
            "role": "user",
            "content": "Alice and Bob are going to a science fair on Friday.",
        },
    ],
    text_format=CalendarEvent,
)

print("response = \n")
pprint(response.output_parsed)

response = 

CalendarEvent(name='Science Fair', date='Friday', participants=['Alice', 'Bob'])


When using Structured Outputs consider also to check for refusals and specify what to do in case of a refusal (i.e. when the model refuses to answer).

Check the official documentation here --> https://platform.openai.com/docs/guides/structured-outputs#refusals

### Difference between `system`, `user`, and other roles in prompt content

**System role:** The `system` message sets the behavior, context, or instructions for the model. It defines how the model should respond and can guide its tone, style, or constraints. For example, you can instruct the model to act as a math tutor or to answer in a specific format.

**User role:** The `user` message represents the actual input or question from the end user. This is the prompt or query you want the model to answer.

**Other roles (e.g., `assistant`):** Some APIs support additional roles like `assistant`, which can be used to provide previous model responses in a conversation, or custom roles for advanced workflows. These help maintain context in multi-turn conversations.

In summary, `system` sets instructions/context, `user` provides the query, and other roles help structure multi-turn or complex interactions.

# Image generation

To generate images with OpenAI APIs, use the `image_generation` tool in your request. Specify your prompt in the `input` field and set the model (e.g., `"gpt-5"`). The API will return a base64-encoded image, which you can decode and save as a file.

In [10]:
from openai import OpenAI
import base64

client = OpenAI() 

response = client.responses.create(
    model="gpt-4o",
    input="Generate an image of gray tabby cat hugging an otter with an orange scarf",
    tools=[{"type": "image_generation"}],
)

# Save the image to a file
image_data = [
    output.result
    for output in response.output
    if output.type == "image_generation_call"
]
    
if image_data:
    image_base64 = image_data[0]
    with open("otter.png", "wb") as f:
        f.write(base64.b64decode(image_base64))

PermissionDeniedError: Error code: 403 - {'error': {'message': 'Your organization must be verified to use the model `gpt-image-1`. Please go to: https://platform.openai.com/settings/organization/general and click on Verify Organization. If you just verified, it can take up to 15 minutes for access to propagate.', 'type': 'invalid_request_error', 'param': None, 'code': None}}